In [1]:
import json
import pandas as pd
import boto3
from decimal import Decimal
import requests
from bs4 import BeautifulSoup
import time
import datetime
from dateutil import parser

dynamodb = boto3.resource('dynamodb', region_name='us-east-2')

In [2]:

def pullDynamo():
    #pull existing data from Dynamodb
    table = dynamodb.Table('usedPlates')
    response = table.scan()
    df = pd.DataFrame(response['Items'])
    return df, df.count()[0]


In [11]:
def pullDynamo():
    #pull existing data from Dynamodb
    table = dynamodb.Table('usedPlates')
    response = table.scan()
    df = pd.DataFrame(response['Items'])
    return df, df.count()[0]

#just to count to see how many listing we're expecting

def pullData():
    # just for initial dynamo run
    existingListingsN = 0
    # pull old data
    newListingsN = 0
    #add something to make use of this, change in N due to process
    #dict to hold the data in
    adict = {"Item Title":[], "Item Price":[], "Listings Collected Number":[], "Sold Date":[]}
    #ebay page url, for loop adds page number
    ebayUrl = "https://www.ebay.co.uk/sch/i.html?_from=R40&_nkw=20kg+weight+plates&_in_kw=1&_ex_kw=&_sacat=0&LH_Sold=1&_udlo=&_udhi=&_samilow=&_samihi=&_sadis=15&_sargn=-1%26saslc%3D1&_fsradio2=%26LH_LocatedIn%3D1&_salic=3&LH_SubLocation=1&_sop=13&_dmd=1&_ipg=60&LH_Complete=&_pgn="
    
    #don't think i need to catch a failed url, as failed url means we've run out of pages

        #range can change to 1 or 2 when automatically pull
    for n in range(1, 5):
        pageUrl = ebayUrl+str(n)
        r= requests.get(pageUrl)
        data=r.text
        soup=BeautifulSoup(data, 'html.parser')
        listings = soup.find_all(class_="s-item s-item__pl-on-bottom")

        for each in listings:
            each_title_span = each.find(class_='s-item__title')
            each_title = each_title_span.next_element.next_element
            # following checks if for each listing, if that listing is already in the database
            #would be good to get a better way of seeing if each item has already been collected, this seems hacky
            #assumes newer listings are at the start of the for loop
            # matching the listing name doesn't work due to package(repeat) sellers, collect purchase id?
            #although presumably not the same issue with used barbells

            #change to if current listing is not equal to top of previous dataframe

            each_price_span = each.find(class_='s-item__price')
            each_price = each_price_span.next_element.next_element

            each_date_span = each.find(class_='s-item__title--tagblock')
            try:
                each_date = parser.parse(each_date_span.next_element.next_element[5:]).isoformat()
            except:
                # sometimes returns a None object
                pass

            #specifying the £ sound to be the first character in item price ensures the html not accidentally picked up there.
            # and we don't want to remove a price without removing the item, so best to exclude the item before it's added to our lists.
            if str(each_price)[:1] == "£":
                existingListingsN +=1
                newListingsN += 1
                adict['Item Title'].append(each_title)
                adict['Item Price'].append(each_price)
                adict['Listings Collected Number'].append(float(existingListingsN))
                adict['Sold Date'].append(each_date)
            else:
                #not sure why I don't get data after some point on the 4th page tried re-writing, oh well
                pass
    
    df = pd.DataFrame(adict)
    if df.count()[0] > 0:
        # returns df of new data to be added
        print("expecting "+str(newListingsN)+" new listings to be added")
        return newListingsN, df
    else:
    # df should only list whatever it collected, ie whatever wasn't already in the csv
        newListingsN = 0
        return newListingsN

    
def dynamoDump():
    #already called dynamo?
    #dynamodb = boto3.resource('dynamodb')
    pullDataOutput = pullData()
    if type(pullDataOutput) == tuple:
        newDF = pullDataOutput[1]
        table = dynamodb.Table('usedPlates')
        #would be better to declare this variable beforehand
        # if newDF == None not working, so will try 
        if isinstance(newDF, pd.DataFrame):
            #if newDF == None:
            #    
            #else:
            if sum(newDF['Listings Collected Number']) > 0:
                n2 = 0
                print("working through here")
                print(newDF)
                with table.batch_writer() as batch:
                    for index, row in newDF.iterrows():
                        n2 += 1
                        batch.put_item(json.loads(row.to_json(), parse_float=Decimal))
                    print("added "+str(n2) + " listing")
            else:
                print("didn't get very far")
        else:
            print("not even close")

In [12]:
dynamoDump()

expecting 236 new listings to be added
working through here
                                            Item Title Item Price  \
0                                        [New listing]     £10.00   
1                                        [New listing]     £35.00   
2    2 x 10kg cast iron weight plates - 1" Hole dia...     £25.00   
3    Adjustable Dumbbells 20Kg 30Kg 50Kg Cast Iron ...     £25.99   
4    Iron Weight Plates Set 5KG 10KG 20KG 1" barbel...     £11.99   
..                                                 ...        ...   
231  150KG 2" Olympic Bumper Weight Plate Set - 5kg...    £270.00   
232  5kg x4 Opti Cast Iron Weights Plates 1" Inch H...     £27.50   
233  Jordan fitness 20kg weight lifting plates gym ...     £40.00   
234  Escape Commercial Tri Grip Olympic Urethane Tr...     £25.00   
235  JLL® Cast Iron Weight Plates, 0.5kg to 20kg pa...     £11.99   

     Listings Collected Number            Sold Date  
0                          1.0  2022-11-27T00:00:00  
1  

ClientError: An error occurred (ValidationException) when calling the BatchWriteItem operation: The provided key element does not match the schema

In [5]:

#ClientError: An error occurred (ValidationException) when calling the BatchWriteItem operation: The provided key element does not match the schema

NameError: name 'newDF' is not defined

In [6]:
table = dynamodb.Table('usedPlates')

In [8]:
float(2)

2.0

In [13]:
time.time()

1669585871.766944